In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In this case, we will try two lane roads. We do a simple case of a two lane roads intersection. Town-Road-On-Ramp intersection. In this case, vehicles are produced at the start of the town road. It moves along the town road until it reaches the end. The intersection is then checked for an available space. If there are available spaces for the vehicle, it is transferred to the intersection. When it reaches the end of intersection, it becomes a candidate of transfer to the on-ramp. If there's avaialable space, it is transferred and moved along the on-ramp. When it reaches the end of the ramp, it is forced to stop to pay for the toll fees. 

In [28]:
class Vehicle:
    def __init__(self, vehicle_id, x_position, y_position, speed, max_speed, length, width, road_designation, vehicle_type):
    #This specified the dimensions and attributes of vehicles. Vehicle identification is also incorporated by assigning IDs
        self.vehicle_id = vehicle_id #for vehicle identification
        self.x_position = x_position #position of the rear bumper
        self.y_position = y_position #lane of the right side of the vehicle
        self.speed = speed #number of cells the vehicle moves per timestep
        self.max_speed = max_speed #maximum speed allowed for a vehicle
        self.braking_prob = 0.01 #probability of braking
        self.length = length #x dimension of vehicle
        self.width = width #y dimension of vehicle
        self.road_designation = road_designation #identify of the road where the vehicle is
        self.vehicle_type = vehicle_type

    def accelerate(self):
    #Accelerate the vehicle by increments of 1 if maximum speed is not yet attained
        if self.speed < self.max_speed:
            self.speed += 1 

    def decelerate(self, gap_distance):
    #Decrease the speed of the vehicle if the gap distance is less than current speed
        if self.speed > gap_distance:
            self.speed = gap_distance

    def randomize(self, braking_prob):
    #Randomly decrease the speed of the vehicle based on braking probability
        if np.random.rand() < braking_prob:
            if self.speed > 0:
                self.speed -= 1

In [48]:
class Road:
    def __init__(self, road_id, road_length, road_width, speed_limit, has_toll_gate = False):
        '''Road features'''
        self.road_id = road_id
        self.road_length = road_length
        self.road_width = road_width
        self.road_occupancy = np.zeros((self.road_length, self.road_width), dtype=int) #initialize a state of road occupancy
        self.vehicles = [] #initialize an empty list of vehicles
        self.vehicle_counter = 0  # To assign unique IDs to vehicles
        self.speed_limit = speed_limit #maximum speed possible
        self.has_toll_gate = has_toll_gate #declares whether a road is an on-ramp or not (boundary condition)

    def update_occupancy(self):
        '''Updates the state of occupancy of the road. 0 means unoccupied, 1 means occupied'''
        self.road_occupancy.fill(0)  # Reset occupancy grid
        for vehicle in self.vehicles:
            if vehicle.x_position < self.road_length and vehicle.y_position < self.road_width:  # Only update occupancy if within road length and width
                x_start = vehicle.x_position
                x_end = min(vehicle.x_position + vehicle.length, self.road_length)
                y_start = vehicle.y_position
                y_end = min(vehicle.y_position + vehicle.width, self.road_width)
                self.road_occupancy[x_start:x_end, y_start:y_end] = 1  # Assuming (y + n) lanes

In [56]:
class IntraRoadSimulator:
    def __init__(self, road):
        self.road = road

    def produce_vehicles(self, production_prob):
        #This method determines the production of new vehicles
        if np.random.rand() < self.production_prob: #Determines the instance/time to produce vehicles
            vehicle_type = np.random.choice(['car', 'jeep', 'truck'], p=[0.3, 0.3, 0.4]) 
            if vehicle_type == 'car':
                vehicle_length = 2
                vehicle_width = 2
            elif vehicle_type == 'jeep':
                vehicle_length = 3
                vehicle_width = 2
            elif vehicle_type == 'truck':
                vehicle_length = 7
                vehicle_width = 2
                
        #Pick a lane for a vehicle produced
        initial_lane = np.random.choice(range(0, self.road.width, 1))

            #Check the availability of the first cells corresponding to vehicle dimensions
        road_portion_checked = self.road_occupancy[0:vehicle_length, initial_lane:initial_lane + vehicle_width]
        if np.sum(road_portion_checked) == 0:  # Check the availability of the portion corresponding to vehicle dimensions
            new_vehicle = Vehicle(self.road.vehicle_counter, 0, initial_lane, np.random.randint(1, self.max_speed + 1), self.max_speed, vehicle_length, vehicle_width, self.road.road_id, vehicle_type)
            self.vehicle_counter += 1
            self.vehicles.append(new_vehicle)
            self.road_occupancy[0:new_vehicle.length, initial_lane:initial_lane + new_vehicle.width] = 1  # Mark the position as occupied

    def gap_distance(self, vehicle): 
        #Determines the distance between the rear bumper of the vehicle ahead and front bumper of the follower vehicle
        current_x_position = vehicle.x_position 
        maximum_possible_distance = self.road_length - current_x_position - vehicle.length #DETERMINES THE CORRESPONDING DISTANCE TO BE CHECKED FOR HEADWAY
        for distance in range(1, maximum_possible_distance + 1): 
            next_x_position = current_x_position + vehicle.length + distance #prospective position of the rear bumper of the vehicle
            if np.any(self.road_occupancy[next_x_position:vehicle.length , initial_lane:vehicle.width] == 1): #checking the space 
                return distance - 1
        return maximum_possible_distance

    def update(self): 
        #updates the road configuration based on nagel-schreckenberg rules
        self.produce_vehicles()
        self.road.update_occupancy()
        for vehicle in self.road.vehicles:
            distance_to_next = self.gap_distance(vehicle)
            vehicle.accelerate()
            vehicle.decelerate(distance_to_next)
            vehicle.randomize(vehicle.braking_prob)
            vehicle.x_position += vehicle.speed

        self.road.update_occupancy()

Let's test the simulation of freeway traffic

Here, I am trying to simulate an intersection. One road before the intersection, and one road after it. 

In [57]:
class InterRoadSimulator:
    def __init__(self, roads):
        #This class facilitates the transfer of vehicles from the road before the intersection to the node, and from the node
        #to the road after the intersection
        self.roads = roads
        self.occupancy_history = []

    def transfer_vehicle(self, vehicle, destination_road):
        #This method facilitates the transfer of a vehicle
        vehicle.x_position = 0 #Reset position to the beginning of the road
        destination_road.vehicles.append(vehicle) #puts the information of the transferred vehicle to the vehicles array of its destination road
        destination_road.update_occupancy() #updates the state of roads occupancy of the destination road

    def handle_toll_gate(self, vehicle, road):
        #Handle tollgate behavior where the vehicles are forced stopped and have a probability of exiting
        if road.has_toll_gate:
            if vehicle.x_position >=road.road_length - 1: #Vehicle reaches the end of the road
                if np.random.rand() < 0.5: #50 % probability of exiting
                    return True
                else:
                    vehicle.speed = 0 #Vehicle stops at the tolll gate
        return False
                
    def update_intersection(self):
        #Define the roads before the intersection, the node road, and the roads after the intersection
        road_before_intersection = self.roads[0]
        node_road = self.roads[1]
        road_after_intersection = self.roads[2]

        #The production of vehicles method is assigned to the road before the intersection
        intra_simulator_before = IntraRoadSimulator(road_before_intersection)
        intra_simulator_before.produce_vehicles()
        
        #Update all roads
        for road in self.roads:
            intra_simulator = IntraRoadSimulator(road)
            intra_simulator.update()

        #Transfer vehicles to the intersection if its distance exceeds the distance of its present road
        vehicles_to_transfer = [vehicle for vehicle in road_before_intersection.vehicles if vehicle.x_position >= road_before_intersection.road_length]
        for vehicle in vehicles_to_transfer:
            self.transfer_vehicle(vehicle, node_road)
        road_before_intersection.vehicles = [vehicle for vehicle in road_before_intersection.vehicles if vehicle.x_position < road_before_intersection.road_length]

        #Transfer vehicles from the intersection to the road after the intersection
        for vehicle in node_road.vehicles[:]:
            if vehicle.x_position >= node_road.road_length:
                initial_lane = np.random.choice([0,1,2]) #starting y position for the other vehicles is always at the rightmost lane(all of them have a width of 2)
                if np.sum(road_after_intersection.road_occupancy[0:vehicle.length, initial_lane:initial_lane+vehicle.width]) == 0: #Check the availability of cells corresponding to vehicle dimensions
                    self.transfer_vehicle(vehicle, road_after_intersection)
                    node_road.vehicles = [vehicle for vehicle in node_road.vehicles if vehicle.vehicle_id != vehicle.vehicle_id]

        #Handle toll gate behavior at the end of the road after the intersection
        vehicles_to_remove = []
        for vehicle in road_after_intersection.vehicles:
            if self.handle_toll_gate(vehicle, road_after_intersection):
                vehicles_to_remove.append(vehicle)
        road_after_intersection.vehicles = [vehicle for vehicle in road_after_intersection.vehicles if v not in vehicles_to_remove]

        self.occupancy_history.append([road.occupancy.copy() for road in self.roads])


        def simulate(self, timesteps):
            for _ in range(timesteps):
                self.update_intersection()

        def save_occupancy_to_csv(self, filename):
            data = []
            for timestep, occupancy in enumerate(self.occupancy_history):
                for road_id, road_occupancy in enumerate(occupancy):
                    flattened_occupancy = road_occupancy.flatten()
                    row = [timestep, road_id] + flattened_occupancy.tolist()
                    data.append(row)
            columns = ['timestep', 'road_id'] + [f'cell_{i}' for i in range(len(flattened_occupancy))]
            df = pd.DataFrame(data, columns=columns)
            df.to_csv(filename, index=False)

In [58]:
# Example of setting up the roads and simulation
road_before_intersection = Road(road_id=0, road_length=100, road_width=4, speed_limit=5)
node_road = Road(road_id=1, road_length=50, road_width = 2, speed_limit=3)
road_after_intersection = Road(road_id=2, road_length=100, road_width=4, speed_limit=5, has_toll_gate=True)

roads = [road_before_intersection, node_road, road_after_intersection]
simulator = InterRoadSimulator(roads)
for _ in range(100):
  simulator.update_intersection()

simulator.save_occupancy_to_csv("occupancy.csv")

TypeError: IntraRoadSimulator.produce_vehicles() missing 1 required positional argument: 'production_prob'